<a href="https://colab.research.google.com/github/yuichi0625/cellar/blob/main/check_country_language_pair_on_transformers/bert_uncased_german.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.0MB 5.7MB/s 
     |████████████████████████████████| 890kB 33.4MB/s 
     |████████████████████████████████| 3.2MB 33.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=2d81dfe5beaa909d257cfe05ff8c98ad4682b18348958c6072a65048b2e3b114
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!wget https://raw.githubusercontent.com/yuichi0625/cellar/main/check_country_language_pair_on_transformers/countries.csv

--2021-03-23 16:31:02--  https://raw.githubusercontent.com/yuichi0625/cellar/main/check_country_language_pair_on_transformers/countries.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7068 (6.9K) [text/plain]
Saving to: ‘countries.csv’

countries.csv       100%[===================>]   6.90K  --.-KB/s    in 0s      

2021-03-23 16:31:03 (66.3 MB/s) - ‘countries.csv’ saved [7068/7068]



In [3]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
pd.set_option('display.max_rows', 500)

tokenizer = AutoTokenizer.from_pretrained('bert-base-german-dbmdz-uncased')
model = AutoModelForMaskedLM.from_pretrained('bert-base-german-dbmdz-uncased')

sequence = 'Ich wohne in {}, und ich spreche {}.'

rows = []
for name in pd.read_csv('countries.csv')['de'].sort_values():
  input = tokenizer.encode(sequence.format(name, tokenizer.mask_token), return_tensors='pt')
  mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

  token_logits = model(input).logits[0]
  mask_token_logits = token_logits[mask_token_index, :][0]
  mask_token_probs = torch.softmax(mask_token_logits, dim=0)

  candidate_indices = torch.topk(mask_token_logits, k=3, dim=0).indices.tolist()
  result = [f'{tokenizer.decode([ci])} ({mask_token_probs[ci] * 100:.0f}%)' for ci in candidate_indices]

  rows.append([name, *result])

Some weights of the model checkpoint at bert-base-german-dbmdz-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
pd.DataFrame(rows, columns=['Land', 'Sprache(top1)', 'Sprache(top2)', 'Sprache(top3)'])

,Land,Sprache(top1),Sprache(top2),Sprache(top3)
0,Afghanistan,englisch (44%),deutsch (23%),russisch (9%)
1,Albanien,deutsch (44%),englisch (19%),italienisch (7%)
2,Algerien,englisch (34%),deutsch (20%),arabisch (9%)
3,Andorra,spanisch (37%),englisch (19%),italienisch (17%)
4,Angola,englisch (49%),deutsch (23%),spanisch (9%)
5,Antigua und Barbuda,englisch (49%),spanisch (28%),deutsch (12%)
6,Argentinien,englisch (31%),deutsch (28%),spanisch (22%)
7,Armenien,russisch (25%),deutsch (22%),englisch (20%)
8,Aserbaidschan,russisch (31%),englisch (25%),deutsch (19%)
9,Australien,englisch (70%),deutsch (17%),spanisch (3%)
